In [2]:
import psycopg2
from psycopg2.extras import execute_values
import faker
import random
from datetime import timedelta

In [3]:
conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

cur.execute("SELECT branch_id FROM branch")

branches = cur.fetchall()
branches = [b[0] for b in branches]
print(branches)

['015cf519-76c9-478e-9bf8-e25bb91f92bb', '8adb1159-cee3-4c59-b08f-3a9aa369d73e', '29e07c3a-47bf-4fef-8fbe-471e1587118c', 'a49d5373-fdae-4e7e-b90f-17344d0a6aaf', 'c26300f2-703c-4022-82c0-b68dc260aa4f', 'ad3cc27c-6b13-46ed-bcfd-cc099f0d5d47', '6d1f53af-555c-4296-9066-e304432d0633', '6feeeb11-f446-46da-9214-cd004151c492', '7192a064-1e01-42c7-87b9-6e6c3d381339', '0e00def4-2be9-405f-bb3e-1a2bc888828b']


In [4]:
conn.rollback()

cur.execute("""
  select employee_id
  from account
  join employee on account.account_id = employee.account_id
  where password = 'User@123'
""")

employee_ids = cur.fetchall()
employee_ids = [e[0] for e in employee_ids]
print(employee_ids)

['99bc598b-52fe-4ff5-87b5-2a2640841337', 'ea327596-24ae-4028-ac5a-aa50a4773cd9', '3e36ea9a-2333-4b7a-a468-75f9fd46334c', '8815e1b0-7b94-42fd-8b74-9a98d71b5465', '38a388f2-0fb2-4211-9a85-3f8eea6f80db', 'b0fa4d83-f315-4999-af1a-07e28607f43e', '3dcca6ae-6f0e-45d6-b4d8-60e1b7d84193', '3f7fa803-87e6-49ce-8ece-a9c7e8ba2061', 'c18fdada-69f3-43e4-b69f-4892a8d961db', 'ca30d484-d2a7-49ad-b964-a710e8777ca3', '49f400ae-7182-4464-a18e-c9101cef6147', 'c94d7242-d6f9-4d07-9625-822f1b74d93c', '247c589d-4eae-4cf3-b8a5-a633cb3702ff', '0bc40079-f8cd-497c-9936-58dc9fdb8764', '8f9f78be-050a-44c3-90f0-258045204502', '82602afe-5f21-4c41-9846-f7711a5b52e9', '4f4e1196-37f0-456a-b30e-051c2a60fead', '6e01ffe8-f204-4ae7-8df8-e61e2a323216', '8aeee04e-254c-4030-bd66-6d1ecf201bbe', '15c233ec-5cc7-4577-9975-09310648adf9', '877c207a-c712-4d0d-a6c2-82599db3e99a', '8d075a6e-e57b-47ef-9bfc-c5c8b96bab14', 'e19b8700-72de-469c-a73e-82b6f4de11a9', 'd078e9b3-6d19-4903-9e80-e6f609ebd852', 'e8c6c784-ffd8-4ab9-96f1-7315047bec0f',

In [ ]:
history = []

for i in range(300):
  branch_id = random.choice(branches)
  employee_id = employee_ids[i]
  start_date = faker.Faker("vi_VN").date_between(start_date='today', end_date='+30d')
  position = random.choice(['staff', 'manager', 'supervisor'])
  salary = random.randint(5000000, 20000000)

  history.append({
    'branch_id': branch_id,
    'employee_id': employee_id,
    'start_date': start_date,
    'position': position,
    'salary': salary
  })
print(len(history))
  

300


In [ ]:
query = """
INSERT INTO employeehistory (employee_id, branch_id, start_date, position, salary)
VALUES %s
"""

values = [
    (a["employee_id"], a["branch_id"], a["start_date"], a["position"], a["salary"])
    for a in history
]

execute_values(cur, query, values, page_size=len(values))

conn.commit()

UniqueViolation: duplicate key value violates unique constraint "employeehistory_pkey"
DETAIL:  Key (branch_id, employee_id, start_date)=(0e00def4-2be9-405f-bb3e-1a2bc888828b, 5b500581-ab06-48f1-922c-ccf8c4d13420, 2025-12-17) already exists.
